# Assignment 10: Dimensionality Reduction

Dataset(s) needed: MNIST ("Modified National Institute of Standards and Technology") dataset.

In [1]:
#Load the MNIST dataset
from scipy.io import loadmat
matfile = loadmat('scikit_learn_data/mldata/mnist-original.mat', squeeze_me=True, struct_as_record=False)
matfile.keys()

dict_keys(['__header__', '__version__', '__globals__', 'mldata_descr_ordering', 'data', 'label'])

<h3> Q.1. Split the data into a training set and a test set (take the first 60,000 instances for training, and the remaining 10,000 for testing).
</h3>

In [11]:
#TODO


NameError: name 'head' is not defined

<h3> Q.2. Train a Logistic Regression classifier on the dataset and see how long it takes.</h3>

In [3]:
from sklearn.linear_model import LogisticRegression
import time

log_clf = #TODO
start_time = #TODO
#TODO: Train the classifier
end_time = #TODO

print("Training took {:.2f}s".format(end_time - start_time))

SyntaxError: invalid syntax (<ipython-input-3-ec811fe52936>, line 4)

<h3> Q.3. Evaluate the resulting model on the test set.</h3>

In [ ]:
#TODO

<h3> Q.4. Use PCA to reduce the dataset's dimensionality, with an explained variance ratio of 95%.</h3>

In [ ]:
#TODO

<h3> Q.5. Train a new Logistic Regression classifier on the reduced dataset and see how long it takes. Was training much faster? Explain your results.
</h3>

In [ ]:
#TODO

<h3> Q.6. Evaluate the new classifier on the test set: how does it compare to the previous classifier? Discuss the speed / accuracy trade-off and in which case you'd prefer a very slight drop in model performance for a x-time speedup in training.
</h3>

In [ ]:
#TODO

<h3> Q.7. Create a new text cell in your Notebook: Complete a 50-100 word summary 
    (or short description of your thinking in applying this week's learning to the solution) 
     of your experience in this assignment. Include:
<br>                                                                    
What was your incoming experience with this model, if any?
what steps you took, what obstacles you encountered.
how you link this exercise to real-world, machine learning problem-solving. (What steps were missing? What else do you need to learn?)
This summary allows your instructor to know how you are doing and allot points for your effort in thinking and planning, and making connections to real-world work.
</h3>

In [ ]:
# Enter summary here